# A journey over APIs for handling geographical data

## OpenStreetMap

http://openstreetmap.org

In [4]:
import sys
print(sys.stdout.encoding)
print(sys.stdin.encoding)


UTF-8
cp1252


## Geocoding and reverse geocoding

1. Geocoding: address or place name -> latitude, longitude
2. Reverse geocoding: latitude, longitude -> readable address or place name

## Nominatim

Nominatim https://wiki.openstreetmap.org/wiki/Nominatim 

Note: Nominatim requires each application to provide their own custom user-agent. See their usage policy: https://operations.osmfoundation.org/policies/nominatim/

Since Nominatim is a regular webservice we can query it using `requests` Python package.

In [5]:
import requests

def _query(url, params):
    # required by Nominatim usage policy
    headers = {
        'User-Agent': 'Pyladies Berlin'
    }
    resp = requests.get(url, params, headers=headers)
    if resp.status_code == requests.codes.ok:
        return resp.json()
    resp.raise_for_status()

# geocoding
def nominatim_search(address):
    address = address.replace(' ', '%20')
    url = 'http://nominatim.openstreetmap.org/search/{}'.format(address)
    params = dict(format='jsonv2', addressdetails=1, limit=1)
    return _query(url, params)
    
# reverse geocoding
def nominatim_reverse(lat, lon):
    url = 'http://nominatim.openstreetmap.org/reverse'
    params = dict(format='jsonv2', lat=lat, lon=lon, addressdetails=1, limit=1)
    return _query(url, params)

### Geocoding

In [6]:
import json
some_json = nominatim_search('potsdamer platz, berlin')
print(json.dumps(some_json, indent=2))
    

[
  {
    "place_id": "69085508",
    "licence": "Data \u00a9 OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright",
    "osm_type": "way",
    "osm_id": "11845212",
    "boundingbox": [
      "52.508628",
      "52.5089126",
      "13.3769102",
      "13.3771842"
    ],
    "lat": "52.5087317",
    "lon": "13.3770476",
    "display_name": "Potsdamer Platz, Mitte, Berlin, 10117, Deutschland",
    "place_rank": "26",
    "category": "highway",
    "type": "secondary",
    "importance": 0.39312074059012,
    "address": {
      "road": "Potsdamer Platz",
      "suburb": "Mitte",
      "city_district": "Mitte",
      "city": "Berlin",
      "state": "Berlin",
      "postcode": "10117",
      "country": "Deutschland",
      "country_code": "de"
    }
  }
]


### Reverse geocoding

In [7]:
lat = '52.5087317'
lon = '13.3770476'

reverse = nominatim_reverse(lat, lon)
print(json.dumps(reverse, indent=2))

{
  "place_id": "3790255",
  "licence": "Data \u00a9 OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright",
  "osm_type": "node",
  "osm_id": "380497986",
  "lat": "52.508862",
  "lon": "13.3771323",
  "place_rank": "30",
  "category": "shop",
  "type": "hairdresser",
  "importance": "0",
  "addresstype": "shop",
  "name": "D. Machts Lounge",
  "display_name": "D. Machts Lounge, Potsdamer Platz, Mitte, Berlin, 10117, Deutschland",
  "address": {
    "hairdresser": "D. Machts Lounge",
    "road": "Potsdamer Platz",
    "suburb": "Mitte",
    "city_district": "Mitte",
    "city": "Berlin",
    "state": "Berlin",
    "postcode": "10117",
    "country": "Deutschland",
    "country_code": "de"
  },
  "boundingbox": [
    "52.508762",
    "52.508962",
    "13.3770323",
    "13.3772323"
  ]
}


## GeoPy

https://geopy.readthedocs.io/en/stable/

### Geocoding

In [14]:
from geopy.geocoders import Nominatim
import json 

geolocator = Nominatim(user_agent="pyladies-berlin")
location = geolocator.geocode(u"Jägerstraße 32, 10117 Berlin", language='en')
print(json.dumps(location.raw, indent=2))

{
  "place_id": "45157935",
  "licence": "Data \u00a9 OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright",
  "osm_type": "node",
  "osm_id": "3213617576",
  "boundingbox": [
    "52.5141169",
    "52.5142169",
    "13.3963086",
    "13.3964086"
  ],
  "lat": "52.5141669",
  "lon": "13.3963586",
  "display_name": "32, J\u00e4gerstra\u00dfe, Spandauer Vorstadt, Mitte, Berlin, 10117, Germany",
  "class": "place",
  "type": "house",
  "importance": 0.44025
}


In [15]:
print(location.address)

32, Jägerstraße, Spandauer Vorstadt, Mitte, Berlin, 10117, Germany


In [16]:
king_latitude = location.latitude
print(king_latitude)

52.5141669


In [17]:
king_longitude = location.longitude
print(king_longitude)

13.3963586


### Reverse geocoding

For reverse geocoding to work with geopy 1.12 we need the hack described in here:
https://github.com/geopy/geopy/issues/262

In [11]:
from geopy.geocoders import Nominatim
from urllib.request import Request

def get_geolocator():
    # Solution from https://github.com/geopy/geopy/issues/262
    geolocator = Nominatim("pyladies-berlin")

    requester = geolocator.urlopen

    def requester_hack(req, **kwargs):
        req = Request(url=req, headers=geolocator.headers)
        return requester(req, **kwargs)

    geolocator.urlopen = requester_hack

    return geolocator

location = get_geolocator().reverse("{}, {}".format(king_latitude, king_longitude))
address = location.address

In [12]:
print(address)

32, Jägerstraße, Spandauer Vorstadt, Mitte, Berlin, 10117, Deutschland


## Overpass API

https://overpass-turbo.eu

1. Nodes -- Musical instrument shops in Berlin
```
node
  [shop=musical_instrument]
  ({{bbox}});
out;
```

2. Ways / Areas -- Beaches around Krumme Lanke
```
way
  [natural=beach]
  ({{bbox}});
(._;>;);
out;
```

3. Relations -- Technische Universitaet Berlin (Ernst-Reuter-Platz)
```
relation
  [amenity=university]
  ({{bbox}});
(._;>;);
out;
```


In [13]:
import overpass
import json
api = overpass.API()
radius = 1500.0
query = 'node(around:{},{},{})["amenity"="ice_cream"]'.format(radius, king_latitude, king_longitude)
print(query)
response = api.get(query)
print('Found {} ice cream places'.format(len(response['features'])))
print(json.dumps(response, indent=2))
features = response['features']

node(around:1500.0,52.5141669,13.3963586)["amenity"="ice_cream"]
Found 6 ice cream places
{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "id": 918134578,
      "geometry": {
        "type": "Point",
        "coordinates": [
          13.3898891,
          52.5075334
        ]
      },
      "properties": {
        "amenity": "ice_cream",
        "ice_cream_counter": "yes",
        "name": "kalter Krieg",
        "shop": "ice_cream",
        "website": "http://www.kalterkrieg-berlin.de/"
      }
    },
    {
      "type": "Feature",
      "id": 960944159,
      "geometry": {
        "type": "Point",
        "coordinates": [
          13.3875824,
          52.5264165
        ]
      },
      "properties": {
        "amenity": "ice_cream",
        "cuisine": "ice_cream",
        "name": "Eishorn"
      }
    },
    {
      "type": "Feature",
      "id": 2033283081,
      "geometry": {
        "type": "Point",
        "coordinates": [
          13.402

What is the straight line distance from King to the ice cream place?

Use the [geodesic distance](https://en.wikipedia.org/wiki/Geodesics_on_an_ellipsoid) which is an ellipsoidal-surface formulae
Alternatively one can use the [great circle distance](https://en.wikipedia.org/wiki/Great-circle_distance) which is a spherical surface formulae of the geographic distance

More on geographical distances [Geographical distance](https://en.wikipedia.org/wiki/Geographical_distance)

In [14]:
from geopy import distance
king = (king_latitude, king_longitude)

class IceCream:
    def __init__(self, lat, lon, name, geodesic_dist):
        self.lat = lat
        self.lon = lon
        self.name = name
        self.geodesic_dist = geodesic_dist
        self.road_dist = float('nan')
    def __str__(self):
        return 'Geodesic dist: {: 4.2f} m\tRoad dist: {: 4.2f} m\t{}'.format(self.geodesic_dist, self.road_dist, self.name)
        
ice_creams = []


for feature in features:
    name = feature['properties'].get('name', 'N/A')
    lat = feature['geometry']['coordinates'][1]
    lon = feature['geometry']['coordinates'][0]
    d = distance.great_circle(king, (lat, lon)).m
    ice_creams.append(IceCream(lat, lon, name, d))
    
ice_creams = sorted(ice_creams, key=lambda x: x.geodesic_dist)
for ice_cream in ice_creams:
    print(ice_cream)

Geodesic dist:  819.67 m	Road dist:  nan m	Fedora Eismanufaktur
Geodesic dist:  858.00 m	Road dist:  nan m	kalter Krieg
Geodesic dist:  1183.56 m	Road dist:  nan m	Waffel oder Becher
Geodesic dist:  1221.24 m	Road dist:  nan m	al teatro
Geodesic dist:  1383.30 m	Road dist:  nan m	Yoli
Geodesic dist:  1486.32 m	Road dist:  nan m	Eishorn


In [15]:
from pyroutelib3 import Router # Import the router
router = Router('foot') # Initialise it

In [16]:
start = router.data.findNode(king_latitude, king_longitude) # Find start and end nodes
print('Start node id in osm is ', start)

Start node id in osm is  29207837


In [17]:
def route_length(route):
    d = 0
    for i in range(len(route)-1):
        d += router.distance(route[i], route[i+1])
    # distance is in km, multiplying by 1000 to get meters
    return d*1000

for ice_cream in ice_creams:
    end = router.data.findNode(ice_cream.lat, ice_cream.lon)
    #print(end)
    status, route = router.doRoute(start, end) # Find the route - a list of OSM nodes

    if status == 'success':
        #routeLatLons = list(map(router.nodeLatLon, route)) # Get actual route coordinates
        #print(routeLatLons)
        ice_cream.road_dist = route_length(route)
        print(ice_cream)


Geodesic dist:  819.67 m	Road dist:  1079.49 m	Fedora Eismanufaktur
Geodesic dist:  858.00 m	Road dist:  1168.23 m	kalter Krieg
Geodesic dist:  1183.56 m	Road dist:  1705.92 m	Waffel oder Becher
Geodesic dist:  1221.24 m	Road dist:  1598.76 m	al teatro
Geodesic dist:  1383.30 m	Road dist:  1701.85 m	Yoli
Geodesic dist:  1486.32 m	Road dist:  1902.63 m	Eishorn
